In [1]:
import pandas as pd

dataMapPath = 'dataSet/data-mapping.csv'
dataMap = pd.read_csv(dataMapPath)
classContents = {}
dataClasses = pd.unique(dataMap['Class'])
for c in dataClasses:
    df = dataMap.loc[dataMap['Class'] == c]
    classContents[c] = df['Content'][:2]

In [2]:
vocabulary_size = 5000
unknown_token = "UNKNOWNTOKEN"
sentence_start_token = "SENTENCESTART"
sentence_end_token = "SENTENCEEND"
new_line_token = "NEWLINE"
separator_token = "SEPARATOR"

In [3]:
from keras.preprocessing.text import text_to_word_sequence
text2word_dict = {}
for cls in classContents:
    text2word_list = list()
    for cont in classContents[cls]:
        text = cont
        text = text.replace('\\', '').replace('\n', ' ' + new_line_token + ' ')
        text = text.replace('--', ' ' + separator_token + ' ')
        end_start_token = sentence_end_token + ' ' + sentence_start_token
        text = text.replace('.', ' ' + end_start_token + ' ')
        text2word = text_to_word_sequence(text, lower=False, )
        text2word_list += text2word
    text2word_dict[cls] = text2word_list

Using TensorFlow backend.


In [4]:
from keras.preprocessing.text import Tokenizer
token = Tokenizer(num_words=vocabulary_size, char_level=False)
mtx_dict = {}
tokens_dict ={}
for c in text2word_dict:
    txt = text2word_dict[c]
    token.fit_on_texts(txt)
    tokens_dict[c] = token
    text_mtx = token.texts_to_matrix(txt, mode='binary')
    mtx_dict[c] = text_mtx

In [5]:
input_output_dict = {}
for c in mtx_dict:
    txt_mtx = mtx_dict[c]
    inpt = txt_mtx[:-1]
    outpt = txt_mtx[1:]
    input_output_dict[c] = (inpt, outpt)

In [6]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
#from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding
#from keras.layers.recurrent import LSTM
from keras.layers.recurrent import SimpleRNN


model_dict = {}
for c in input_output_dict:
    inpt, outpt = input_output_dict[c]
    model = Sequential()
    model.add(Embedding(input_dim=inpt.shape[1],output_dim= 42, input_length=inpt.shape[1]))
    model.add(SimpleRNN(256, activation='relu'))
    model.add(Dense(outpt.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=["accuracy"])
    model.fit(inpt, y=outpt, batch_size=300, epochs=10, verbose=1, validation_split=0.2)
    model_dict[c] = model

Train on 1800 samples, validate on 451 samples
Epoch 1/10


 300/1800 [====>.........................] - ETA: 45s - loss: 6.9084 - acc: 0.0000e+00

 600/1800 [=========>....................] - ETA: 33s - loss: 6.9048 - acc: 0.0000e+00

 900/1800 [==============>...............] - ETA: 24s - loss: 6.8996 - acc: 0.0178    

1200/1800 [===================>..........] - ETA: 16s - loss: 6.8893 - acc: 0.0308

...

 300/1420 [=====>........................] - ETA: 28s - loss: nan - acc: 0.2533

 600/1420 [===========>..................] - ETA: 20s - loss: nan - acc: 0.2483

 900/1420 [==================>...........] - ETA: 13s - loss: nan - acc: 0.2567

1200/1420 [========================>.....] - ETA: 5s - loss: nan - acc: 0.2567 

1420/1420 [==============================] - 40s - loss: nan - acc: 0.2599 - val_loss: nan - val_acc: 0.2247

In [28]:
import numpy as np


def get_next_word(text, token, model, fullmtx, fulltext):
    tmp = text_to_word_sequence(text, lower=False, split=" ")
    tmp = token.texts_to_matrix(tmp, mode='binary')
    p = model.predict(tmp)
    bestmatch = np.min(np.argmax(p))
    candidates = np.where(fullmtx[:,bestmatch]>0)
    try:
        next_idx = np.min(candidates)
        return fulltext[next_idx]
    except ValueError:
        return unknown_token

In [30]:
for c in dataClasses:
    mtx = mtx_dict[c]
    m = model_dict[c]
    fulltxt = text2word_dict[c]
    listoftext = np.unique(fulltxt)
    token = tokens_dict[c]
    seq_txt = ''
    for txt in listoftext:
        last_gen = get_next_word(txt, token, m, mtx, fulltxt)
        if last_gen != unknown_token:
            fulltxt.remove(last_gen)
            seq_txt += ' ' + last_gen
    print(c)
    print(seq_txt)
    print()

business
 Cantona and Ole Gunnar Solskjaer and club manager Sir Alex Ferguson have lent their backing to the supporters' group Shareholders United SENTENCEEND SENTENCESTART They have all spoken out against the bid SENTENCEEND SENTENCESTART A spokesman for the supporters club said I can't see any difference compared to Mr Glazer's previous proposals other than £200m less debt SENTENCEEND SENTENCESTART He isn't bringing any money into the club he'll use our money to buy it SENTENCEEND SENTENCESTART Mr Glazer's latest move is being led by Mr Glazer's two sons Avi and Joel according to the Financial Times SENTENCEEND SENTENCESTART A proposal was received by David Gill United's chief executive at the end of last week pitched at about 300p a share SENTENCEEND SENTENCESTART David Cummings head of UK equities for Standard Life Investments said he believed a well funded 300p a share bid would be enough for Mr Glazer to take control of the club SENTENCEEND SENTENCESTART I do not think there is a

tech




sport
 SENTENCESTART WRU chairman David Pickering said the structure would end problems of player availability for club and country SENTENCEEND SENTENCESTART He added We feel sure that spectator interest would respond to the impetus of high intensity rugby being played continuously rather than the fragmented timetable currently in operation SENTENCEEND SENTENCESTART Equally we suspect that the sponsors would prefer the sustained interest in a continuous tournament and hopefully the broadcasters would also enjoy increased exposure SENTENCEEND SENTENCESTART Moving the Six Nations from its traditional February beginning should also ensure better weather conditions and stimulate greater interest in the games and generally provide increased skills and competition and attract greater spectator viewing Pickering argued SENTENCEEND SENTENCESTART The plan will be put before the International Rugby Board next month where four other plans drawn up by independent consultants for a global integrate

politics




entertainment


